# The Viewer Component
The interactive viewer for IFC models has been extended from Thomas Paviots' excellent [JupyterRenderer](https://github.com/tpaviot/pythonocc-core/blob/master/src/Display/WebGl/jupyter_renderer.py) and offeres a number of functionalities:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ifcopenshell
import json
import collections
import math
import uuid

import numpy as np
import pymeshlab as ml
from tqdm import tqdm
from ifcopenshell.util.selector import Selector
from ipywidgets import interact
from compas.geometry import oriented_bounding_box_numpy

from OCC.Core.Bnd import Bnd_Box, Bnd_OBB
from OCC.Core.BRepBndLib import brepbndlib_AddOBB
from OCC.Core.BRepPrimAPI import (BRepPrimAPI_MakeBox, 
BRepPrimAPI_MakeSphere, BRepPrimAPI_MakeCylinder)
from OCC.Core.gp import gp_Pnt, gp_XYZ, gp_Ax2, gp_Dir

from utils.JupyterIFCRenderer import JupyterIFCRenderer

In [3]:
m = ifcopenshell.open("../merged_full.ifc")
system_dict_file = "../WestDeckBox.nwd_aggregation.json"
#m = ifcopenshell.open("data/231110AC-11-Smiley-West-04-07-2007.ifc")k

## Relationship Identification

###  aggrgegation relationships

In [42]:
# flatten info dictionary
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        #print(type(v))
        if isinstance(v, list):
            for i, el in enumerate(v):
                new_key_2 = new_key + sep + str(i)
                if isinstance(el, collections.MutableMapping):
                    items.extend(flatten(el, new_key_2, sep=sep))
                else:
                    
                    items.append(el)
        elif isinstance(v, collections.MutableMapping):
            #print(v)
            items.extend(flatten(v, new_key, sep=sep))
        else:
            print(type(v))
            items.append(v)
    return (items)


def getSystems(system_dict_file):
    f = open(system_dict_file)
    system_dict = json.load(f)
    root = list(system_dict.keys())[0]
    out_dict = {}

    for sys in system_dict[root]:
        sys_name = list(sys.keys())[0]
        
        out_dict[sys_name] = flatten(sys)
        print(sys_name, len(out_dict[sys_name]))
    return out_dict
        

getSystems(system_dict_file)    
    
    

/tmp/ipykernel_581/2049937458.py:9: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if isinstance(el, collections.MutableMapping):


West-DeckBox-Support.rvm 57062
West-DeckBox-Electrical.rvm 32334
West-DeckBox-Structure.rvm 19892
West-DeckBox-Instrumentation.rvm 37549
West-DeckBox-Pipe.rvm 33565
West-DeckBox-Mechanical.rvm 31107
West-DeckBox-Architecture.rvm 11181
West-DeckBox-Safety.rvm 2638
West-DeckBox-HVAC.rvm 7749


{'West-DeckBox-Support.rvm': ['PVOLUME 1 of STRUCTURE /480-PSU-7168',
  'SCTN 1 of FRMWORK /480-PSU-7168/MAIN',
  'SCTN 2 of FRMWORK /480-PSU-7168/MAIN',
  'PVOLUME 1 of STRUCTURE /480-PSU-7289',
  'SCTN 1 of FRMWORK /480-PSU-7289/MAIN',
  '',
  'FITTING 1 of SCTN 2 of FRMWORK /480-PSU-7289/MAIN',
  'PVOLUME 1 of STRUCTURE /481-PSU-5777',
  'SCTN 1 of FRMWORK /481-PSU-5777/MAIN',
  '',
  'FITTING 1 of SCTN 2 of FRMWORK /481-PSU-5777/MAIN',
  'PVOLUME 1 of STRUCTURE /481-PSU-5778',
  'SCTN 1 of FRMWORK /481-PSU-5778/MAIN',
  '',
  'FITTING 1 of SCTN 2 of FRMWORK /481-PSU-5778/MAIN',
  'PVOLUME 1 of STRUCTURE /481-PSU-5779',
  'SCTN 1 of FRMWORK /481-PSU-5779/MAIN',
  '',
  'FITTING 1 of SCTN 2 of FRMWORK /481-PSU-5779/MAIN',
  'PVOLUME 1 of STRUCTURE /481-PSU-5780',
  'SCTN 1 of FRMWORK /481-PSU-5780/MAIN',
  '',
  'FITTING 1 of SCTN 2 of FRMWORK /481-PSU-5780/MAIN',
  'PVOLUME 1 of STRUCTURE /481-PSU-5781',
  '',
  'FITTING 2 of SCTN 1 of FRMWORK /481-PSU-5781/MAIN',
  'SCTN 2 of FRMWO

### Topological relationships

## Visualization

In [80]:
viewer = JupyterIFCRenderer(m, size=(400,300))
viewer.setAllTransparent()
viewer

RuntimeError: Failed to process shape

### aggregation relationships

In [15]:
picker = viewer.colorPicker()
picker


ColorPicker(value='blue', description='Pick a color')

In [6]:
out_dict = {'HVAC':['Rohrtypen:Kupfer - Hartgelötet:7718868', 'Rohrtypen:Kupfer - Hartgelötet:7718886', ],
           'electrical':[ 'Rohrtypen:Kupfer - Hartgelötet:7718872', 'Rohrtypen:Kupfer - Hartgelötet:7718880']}

# PAINT A SET OF ELEMENTS IN ONE COLOUR
def systemSelect(system):
    selector = Selector()
    for e in out_dict[system]:
        element = selector.parse(m, '.IfcProduct[Name *= "' + e + '"]')[0]
        viewer.setColor(element, picker.value)
    return system

interact(systemSelect, system=['HVAC', 'electrical'])

interactive(children=(Dropdown(description='system', options=('HVAC', 'electrical'), value='HVAC'), Output()),…

<function __main__.systemSelect(system)>

Instances of building elements with represenations can be selected interactivly. Information such as the attributes `GUID`, `Name` etc. are displayed to the left of the 3D viewport.

In [211]:
# reset colours
viewer.setDefaultColors()

### topological relationships



## replace jupyter renderer

1. Compute the bounding box of ifc product directly from points
2. generate ifc elements to indicate relationships


In [6]:
element_name = "TUBE 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
element_type = "IFCPIPESEGMENT"

selector = Selector()
element = selector.parse(
    m, '.' + element_type + '[Name *= "' + element_name + '"]')[0]

shape = element.Representation.Representations[0].Items[0]
element_coords = np.array(shape.Coordinates.CoordList)
#print(element_coords)
bbox = oriented_bounding_box_numpy(element_coords)
print(bbox)

[[-44.38016913  10.50333148  53.12546917]
 [-44.34301252  10.53382514  53.12546917]
 [-44.34301252  10.53382514  53.94964983]
 [-44.38016913  10.50333148  53.94964983]
 [-44.34967547  10.46617486  53.12546917]
 [-44.31251886  10.49666852  53.12546917]
 [-44.31251886  10.49666852  53.94964983]
 [-44.34967547  10.46617486  53.94964983]]


In [4]:
create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)
owner_history = m.by_type("IfcOwnerHistory")[0]
project = m.by_type("IfcProject")[0]
context = m.by_type("IfcGeometricRepresentationContext")[0]
floor = m.by_type("IfcBuildingStorey")[0]

# get bounding box of ifc element
def get_oriented_bbox(element_name, element_type):
    # get element and bbox
    selector = Selector()
    element = selector.parse(
        m, '.' + element_type + '[Name *= "' + element_name + '"]')[0]

    #print(element.Representation.Representations)
    shape = element.Representation.Representations[0].Items[0]
    element_coords = np.array(shape.Coordinates.CoordList)
    #print(element_coords)
    bbox = oriented_bounding_box_numpy(element_coords)
    
    # identify box orientation
    l1 = sq_distance(bbox[0][0], bbox[0][1], bbox[0][2], bbox[1][0], bbox[1][1], bbox[1][2])
    l2 = sq_distance(bbox[0][0], bbox[0][1], bbox[0][2], bbox[3][0], bbox[3][1], bbox[3][2])
    l3 = sq_distance(bbox[0][0], bbox[0][1], bbox[0][2], bbox[4][0], bbox[4][1], bbox[4][2])
    half_lengths = [l1, l2, l3]
    
    dominant_axis = half_lengths.index(max(half_lengths))
    if dominant_axis == 0:
        dominant_direction = [bbox[0][i] - bbox[1][i] for i in range(3)]
    elif dominant_axis == 1:
        dominant_direction = [bbox[0][i] - bbox[3][i] for i in range(3)]
    else:
        dominant_direction = [bbox[0][i] - bbox[4][i] for i in range(3)]

    dominance_ratio = max(half_lengths)/sorted(half_lengths)[-2]
    center = [(bbox[0][i] + bbox[6][i])/2 for i in range(3)]

    #print(dominance_ratio, dominant_direction)
    
    #print(center)
    return([dominant_direction, dominance_ratio, 
           half_lengths, center])


def CreateBeam(ifcFile, container, name, section, L, position, direction, owner_history, context):
    Z = 0.,0.,1.
    print('length', L)
    B1 = ifcFile.createIfcBeam(create_guid(),owner_history , name)
    B1.ObjectType ='beam'
    
    print(type(position[0]))
    B1_Point =ifcFile.createIfcCartesianPoint ( tuple(position) ) 
    #B1_Point =ifcFile.createIfcCartesianPoint ( (0.0,0.0,0.0) ) 
    B1_Axis2Placement = ifcFile.createIfcAxis2Placement3D(B1_Point)
    B1_Axis2Placement.Axis = ifcFile.createIfcDirection(direction)
    B1_Axis2Placement.RefDirection =ifcFile.createIfcDirection(np.cross(direction, Z).tolist())

    B1_Placement = ifcFile.createIfcLocalPlacement(container.ObjectPlacement,B1_Axis2Placement)
    B1.ObjectPlacement=B1_Placement
    B1_ExtrudePlacement = ifcFile.createIfcAxis2Placement3D(ifcFile.createIfcCartesianPoint ( (0.,0.,0.) )   )
   
    B1_Extruded=ifcFile.createIfcExtrudedAreaSolid()
    B1_Extruded.SweptArea=section
    B1_Extruded.Position=B1_ExtrudePlacement
    B1_Extruded.ExtrudedDirection = ifcFile.createIfcDirection(Z)
    B1_Extruded.Depth = L

    B1_Repr=ifcFile.createIfcShapeRepresentation()
    B1_Repr.ContextOfItems=context
    B1_Repr.RepresentationIdentifier = 'Body'
    B1_Repr.RepresentationType = 'SweptSolid'
    B1_Repr.Items = [B1_Extruded]
    
    B1_DefShape=ifcFile.createIfcProductDefinitionShape()
    B1_DefShape.Representations=[B1_Repr]
    B1.Representation=B1_DefShape
    
    Flr1_Container = ifcFile.createIfcRelContainedInSpatialStructure(create_guid(),owner_history)
    Flr1_Container.RelatedElements=[B1]
    Flr1_Container.RelatingStructure= container
    
    
def Circle_Section(r, ifcfile):
    B1_Axis2Placement2D =ifcfile.createIfcAxis2Placement2D( 
                          ifcfile.createIfcCartesianPoint( (0.,0.,0.) ) )
    B1_AreaProfile = ifcfile.createIfcCircleProfileDef("AREA")
    B1_AreaProfile.Position = B1_Axis2Placement2D 
    B1_AreaProfile.Radius = r
    return B1_AreaProfile


#get_oriented_bbox(element_name, element_type)

NameError: name 'element_name' is not defined

In [6]:
# get bounding box of ifc element
# def get_oriented_bbox(element_name, element_type):
#     # select shape
#     selector = Selector()
#     element = selector.parse(
#         m, '.' + element_type + '[Name *= "' + element_name + '"]')[0]
#     # element_coords = element.Representation.Representations[0].Items[0].Coordinates.CoordList
#     shape = list(viewer.shapedict.keys())[list(
#         viewer.shapedict.values()).index(element)]
#     # o_bbox = get_oriented_boundingbox(shape)
    
#     # get oriented bbox
#     obb = Bnd_OBB()
#     is_triangulation_used = True
#     is_optimal = True
#     is_shape_tolerance_used = False
#     brepbndlib_AddOBB(
#         shape, obb, is_triangulation_used, is_optimal, 
#         is_shape_tolerance_used)
    
#     # identify box orientation
#     half_lengths = [obb.XHSize(), obb.YHSize(), obb.ZHSize()]
#     dominant_axis = half_lengths.index(max(half_lengths))
#     if dominant_axis == 0:
#         dominant_direction = obb.XDirection()
#     elif dominant_axis == 1:
#         dominant_direction = obb.YDirection()
#     else:
#         dominant_direction = obb.ZDirection()
#     dominance_ratio = max(half_lengths)/sorted(half_lengths)[-2]
    
#     return([dominant_direction, dominance_ratio, 
#            half_lengths, obb.Center()])


def get_point_along_axis(init_point, axis, half_length, edge_distance):
    #print(half_length)
    return (init_point + axis*(half_length - edge_distance))


def sq_distance(x1, y1, z1, x2, y2, z2):
    return ((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)


# identify a candidate for an edge of the relationship visualization
def get_corner(obb, center_other, edge_distance):
    direction = obb[0]
    center = obb[3]

    candidate1 = [get_point_along_axis(center[i], direction[i], 
                                    obb[2][i], edge_distance) 
               for i in range(3)]
    candidate2 = [get_point_along_axis(center[i], direction[i], 
                                    -1*obb[2][i], -1*edge_distance) 
               for i in range(3)]
    # print(corner1, corner2)
    
    dist1 = sq_distance(center_other[0], center_other[1],
                   center_other[2], candidate1[0],
                   candidate1[1], candidate1[2])    
    dist2 = sq_distance(center_other[0], center_other[1],
                   center_other[2], candidate2[0],
                   candidate2[1], candidate2[2]) 
    
    if dist1 < dist2:
        return candidate1
    else:
        return candidate2


def draw_sphere(point, radius, colour, viewer):
    point = gp_Pnt(point)
    ball = BRepPrimAPI_MakeSphere(point, radius).Shape()
    viewer.DisplayShape(ball, shape_color = colour, 
                        transparency=True, opacity=0.8)


def draw_cylinder(p1, p2, radius, colour,  element_name1, element_name2, ifc):
    sectionC1 = Circle_Section(r=radius, ifcfile=ifc)
    name='rel '+element_name1 + ' x ' + element_name2
    ConnectingBeam_1 = CreateBeam(ifc, container=floor, name=name, 
                                  section= sectionC1, 
                                  L=math.sqrt(sq_distance(p1[0],p1[1],p1[2],p2[0],p2[1],p2[2])),
                                  position=([p2[0].item(), p2[1].item(), p2[2].item()]),
                                  direction=((p1[0] - p2[0]).item(), (p1[1] - p2[1]).item(), 
                                             (p1[2] - p2[2]).item()),
                                  owner_history=owner_history, context=context)
#     ifc.create_entity('IfcPipeSegment', GlobalId=ifcopenshell.guid.new(), 
#                       Name='rel '+element_name1 + ' x ' + element_name2)
    
# def draw_cylinder(p1, p2, radius, colour, viewer):
#     x = (p1.X() + p2.X())/2
#     y = (p1.Y() + p2.Y())/2
#     z = (p1.Z() + p2.Z())/2
#     p = gp_Pnt(p2.X(), p2.Y(), p2.Z())
#     v = gp_Dir((p1.X() - p2.X()), (p1.Y() - p2.Y()), (p1.Z() - p2.Z()))
#     ax = gp_Ax2(p, v)
#     h = math.sqrt(sq_distance(p1.X(), p1.Y(), p2.Z(), 
#                               p2.X(), p2.Y(), p2.Z()))
#     print(x,y,z,p,v,ax,h)
    
#     cylinder = BRepPrimAPI_MakeCylinder(ax, radius, h).Shape()
#     viewer.DisplayShape(cylinder, shape_color = colour, 
#                         transparency=True, opacity=0.8)
    

# draw a visual indication of a topological relationship    
def draw_relationship (element_name1, element_type1, 
                       element_name2, element_type2, ifc):
    # define params
    #TODO: dynamically scale sphere_radius
    sphere_radius = 0.03
    colour = 'blue'
    threshold = 3
    edge_distance = 0.05
    
    # get bboxes
    obb1 = get_oriented_bbox(element_name1, element_type1)
    obb2 = get_oriented_bbox(element_name2, element_type2)
    
    # if bboxes are roughly square, then use centerpoint, 
    # otherwise use a point closer to the edge
    # NOTE: algorithm might fail if dominance ratio is high but
    # the dominent length is still lower than edge_distance
    #print(obb2[2], obb1[2])
    #print('dist', obb1[1])
    if obb1[1] < threshold:
        corner1 = obb1[3]
    else:
        corner1 = get_corner(obb1, obb2[3], edge_distance)
    if  obb2[1] < threshold:
        corner2 = obb2[3]
    else:
        corner2 = get_corner(obb2, obb1[3], edge_distance)
    #print(corner1.X(),corner1.Y(),corner1.Z(),corner2.X(),corner2.Y(),corner1.Z())
    draw_cylinder(corner1, corner2, sphere_radius, 'green', element_name1, element_name2, ifc)
#     draw_sphere(corner1, sphere_radius, colour, viewer)
#     draw_sphere(corner2, sphere_radius, 'red', viewer)
 

In [7]:
element1_name = "TUBE 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
element1_type = "IFCPIPESEGMENT"
element2_name = "ELBOW 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
element2_type = "IFCPIPEFITTING"

element3_name = "TUBE 2 of BRANCH /AM-8120227-WD-MDA-01/B1"
element3_type = "IFCPIPESEGMENT"
element4_name = "ELBOW 2 of BRANCH /AM-8120227-WD-MDA-01/B1"
element4_type = "IFCPIPEFITTING"
element5_name = "TUBE 3 of BRANCH /AM-8120227-WD-MDA-01/B1"
element5_type = "IFCPIPESEGMENT"

draw_relationship(element1_name, element1_type, 
                  element2_name, element2_type, m)
draw_relationship(element2_name, element2_type, 
                  element3_name, element3_type, m)
draw_relationship(element3_name, element3_type, 
                  element4_name, element4_type, m)
draw_relationship(element4_name, element4_type, 
                  element5_name, element5_type, m)
#element1_center, element1_coords = get_element_deets()



# centerpoint =gp_Pnt(element1_center)
# ball = BRepPrimAPI_MakeSphere(centerpoint, 0.02).Shape()

length 0.3997386749277167
<class 'float'>
length 0.125058685320813
<class 'float'>
length 0.12506249509751077
<class 'float'>
length 0.20628940752890432
<class 'float'>


In [8]:
m.write('../edited.ifc') 

## Misc

In [128]:
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)
pdct_shape = ifcopenshell.geom.create_shape(settings, inst=element)
viewer.DisplayShape(ball, shape_color = 'blue', transparency=False, opacity=0.5)

In [92]:
o_bbox1 = get_oriented_boundingbox(list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element1)])
bbox_center1 = [o_bbox[0].X(), o_bbox[0].Y(), o_bbox[0].Z()]


-44.346343994140646

In [66]:
viewer.shapedict.values()

dict_values([#44265=IfcPipeSegment('1VZtjMEa9OyHYWQ2DO1_t0',#44264,'TUBE 6 of BRANCH /AM-8120227-WD-MDA-01/B1',$,$,#44252,#44253,$,$), #44281=IfcPipeSegment('1THpb6WcUCe1wMJvf3AQ9K',#44264,'TUBE 5 of BRANCH /AM-8120227-WD-MDA-01/B1',$,$,#44274,#44275,$,$), #44297=IfcPipeSegment('0k6a9u4GxbQCqRUeqZTppc',#44264,'TUBE 4 of BRANCH /AM-8120227-WD-MDA-01/B1',$,$,#44290,#44291,$,$), #44313=IfcPipeSegment('3n5tqnHGXGsWlFINWBVpM$',#44264,'TUBE 3 of BRANCH /AM-8120227-WD-MDA-01/B1',$,$,#44306,#44307,$,$), #44329=IfcPipeSegment('2YC3hxwEFSgxzZyAggsmRa',#44264,'TUBE 2 of BRANCH /AM-8120227-WD-MDA-01/B1',$,$,#44322,#44323,$,$), #44345=IfcPipeSegment('2sfswrTwhvzW2BoJOlgpMN',#44264,'TUBE 1 of BRANCH /AM-8120227-WD-MDA-01/B1',$,$,#44338,#44339,$,$), #88909=IfcPipeSegment('1GnilcLysWc5tz3r7HJDvS',#44264,'/WD/MDB/HD/I/PEN/L/TUBE/HW',$,$,#88902,#88903,$,$), #162402=IfcPipeFitting('3Nr0h2wMzPTwedeCcR$sSG',#159832,'FLANGE 1 of BRANCH /AM-8120227-WD-MDA-01/B1',$,$,#162394,#162401,$,$), #162412=IfcPipeFitti

In [78]:
list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element)]

<class 'TopoDS_Compound'>

In [73]:
viewer._shapes

{'41ae28d88c44404daa65545c590ec015': <class 'TopoDS_Compound'>,
 'f294530a01164620ab765cae4202420d': <class 'TopoDS_Compound'>,
 '044b0a8cb1ba4b03aa0a5dd1d52436be': <class 'TopoDS_Compound'>,
 'c49c5eec4340495cb5733be28edf6df4': <class 'TopoDS_Compound'>,
 '59307f7e390145a5ae7868014fb887f2': <class 'TopoDS_Compound'>,
 '3d019c765c0141edb285e2df71448f62': <class 'TopoDS_Compound'>,
 '8cacb9a6c4e7493281c4302a754e36e3': <class 'TopoDS_Compound'>,
 '5b58d7d57eff42d19c60c4fd17803132': <class 'TopoDS_Compound'>,
 '64c16522104f474fa8faf7232a95ed81': <class 'TopoDS_Compound'>,
 'f9f08bc32f004c4b8900a52f2e8af557': <class 'TopoDS_Compound'>,
 '64595686f59b4e13baa3baee41f9c5c6': <class 'TopoDS_Compound'>,
 '2a9ac17d91cd4e21b2716f6bec5711b9': <class 'TopoDS_Compound'>,
 '3204622c10134dc3a7b7a2cdcd36e586': <class 'TopoDS_Compound'>,
 '8f6d6b25af6842c28d8a8a45baf5c9bc': <class 'TopoDS_Compound'>,
 '44b92d9c5aee4a93b6aa420652933294': <class 'TopoDS_Compound'>}

In [86]:
BoundingBox([[list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element)]]]).xmin

-44.38697771227844

In [88]:
[list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element)]]

[<class 'TopoDS_Compound'>]

In [32]:
print(len(element.Representation.Representations[0].Items[0].CoordIndex))

128


In [39]:
element.Representation.Representations[0].Items[0].CoordIndex = element.Representation.Representations[0].Items[0].CoordIndex[:1]

## Selecting instances 

The instance object currently selected in the 3D view can be assigned to a avariable using the `.getSelectedProduct()` method. 

Provding the plain varialbe name call sthe `Display()`-method of the instance and prints the line of the original ifc-Datei the  [SPFF-format](https://en.wikipedia.org/wiki/ISO_10303-21).


In [47]:
selection = viewer.getSelectedProduct()
selection

#217510=IfcPipeSegment('3ELBYKoMX93QrK7TjQe0eV',#42,'Rohrtypen:Kupfer - Hartgelötet:7718880',$,'Rohrtypen:Kupfer - Hartgelötet',#217494,#217507,'7718880',.NOTDEFINED.)

## Show / Hide Objects
Programatically or interactively selected projects can be hidden or shown in the viewer using `setVisible()`.


In [ ]:
viewer.setVisible(selection, False)
viewer

Opening elements are displayed by default. Let's hide them.

In [ ]:
for opening in m.by_type("IfcOpeningElement"):
    viewer.setVisible(opening, False)

## Coloring Objects
Objects can be colored either by hexadicimal numbers provided as strings as they are common in e.g. in HTML and CSS and can be retrieved from many ressources

In [55]:

viewer.setColorSelected("#ddffaa")
viewer

AttributeError: 'NoneType' object has no attribute 'material'

If you do not happen to have a color in mind  just call a color picker from the viewer:

In [7]:
picker = viewer.colorPicker()
picker

ColorPicker(value='blue', description='Pick a color')

In [ ]:
viewer.setColorSelected(picker.value)

### Downsampling



In [163]:
ifc = ifcopenshell.open("../deckboxtee_down.ifc")


In [158]:
element_type = 'IFCPIPEFITTING'
selector = Selector()
tees = selector.parse(ifc, '.' + element_type)
print(tees[0])

#802=IfcPipeFitting('2qYdtMmVJo2ogZK_9uUHCy',#2,'TEE 5 of BRANCH /PNL-6160345-H6-TP3/B4',$,$,#813,#812,$,$)


In [162]:



for element in tqdm(tees):
    # create pymeshlab mesh
    #print(element)
    shape = element.Representation.Representations[0].Items[0]
    element_faces = [np.array([i[0]-1, i[1]-1, i[2]-1]) for i in shape.CoordIndex]
    element_coords = np.array(shape.Coordinates.CoordList)
    print(len(element_faces))
#     mesh = ml.Mesh(element_coords, element_faces)
#     ms = ml.MeshSet()
#     ms.add_mesh(mesh, "x")
#     #ms.save_current_mesh("../output.ply")
    
#     # downsample and reassign
#     ms.apply_filter('simplification_clustering_decimation', threshold=ml.Percentage(2))
#     m = ms.current_mesh()
#     shape.CoordIndex = m.face_matrix().tolist()
#     shape.Coordinates.CoordList = m.vertex_matrix().tolist()
    

 25%|████████████████████████▋                                                                         | 224/891 [00:00<00:00, 742.96it/s]

700
492
444
700
256
256
514
752
256
254
256
256
780
254
248
248
256
256
256
756
248
836
692
700
708
720
700
256
256
256
256
256
248
444
444
444
256
256
256
720
700
700
256
768
256
256
216
198
256
444
700
256
852
256
248
256
256
780
700
444
248
248
256
240
852
256
700
256
256
708
256
256
256
256
812
444
444
256
752
240
240
256
256
256
700
588
256
256
256
248
256
438
248
256
256
256
780
692
444
240
780
588
256
256
256
254
752
700
256
240
256
256
256
444
444
248
692
444
444
852
248
248
248
256
240
256
752
700
700
256
256
256
256
692
248
256
692
692
700
256
256
256
248
256
248
256
240
256
256
676
700
780
420
654
700
240
256
256
256
256
248
768
752
240
256
240
256
700
256
256
700
384
444
256
256
256
254
240
256
768
256
444
444
256
152
444
256
256
256
240
256
256
720
444
588
700
780
444
476
256
248
248
256
256
252
256
256
768
240
248
256
256
444
540
700
444
444
444
852
256
256
256
720
768


 54%|████████████████████████████████████████████████████▋                                             | 479/891 [00:00<00:00, 777.58it/s]

252
256
256
248
444
700
256
444
436
700
256
256
256
256
720
216
752
256
256
256
248
248
700
256
852
700
700
768
256
852
700
256
768
240
256
256
256
256
256
724
716
784
700
444
240
700
692
256
218
240
752
256
248
216
256
240
256
248
422
692
444
248
716
752
256
254
852
256
256
768
588
444
700
248
444
444
256
752
256
218
256
256
248
248
256
240
256
256
752
756
692
700
256
692
444
700
256
256
720
256
256
256
256
256
248
256
752
256
256
444
700
100
700
256
256
256
248
256
256
444
692
700
836
248
240
236
256
700
780
700
720
256
256
256
256
444
700
240
256
256
256
256
240
256
444
444
768
256
444
256
256
256
780
256
256
256
256
256
248
240
720
256
248
240
780
700
492
256
444
256
256
256
256
256
700
256
752
256
248
692
444
444
256
444
444
852
248
256
256
240
256
256
700
760
240
692
444
256
444
692
676
256
256
248
256
232
256
248
720
256
700
256
256
444
700
700
444
588
700
256
256
256
248
256
256
240
256
720
256
796
716
700
256
256
692
700
256
256
256
256
256
240
240
256
218
256
256
256
760
700


 84%|██████████████████████████████████████████████████████████████████████████████████▋               | 752/891 [00:00<00:00, 797.08it/s]

256
238
256
256
756
780
588
444
756
692
407
444
256
256
256
218
248
232
256
256
720
248
248
444
444
444
411
588
588
256
256
256
256
248
752
244
752
256
256
444
436
700
444
436
700
256
216
256
256
256
256
752
232
240
256
700
700
852
444
700
700
256
248
256
256
240
256
248
248
256
256
720
256
240
700
444
256
692
700
418
256
256
236
256
256
256
248
256
256
248
768
256
216
256
852
780
700
588
248
760
256
256
240
256
256
256
218
768
744
444
692
444
444
720
240
256
256
418
256
256
256
256
720
720
768
444
444
700
492
692
716
256
248
256
256
256
256
780
256
256
256
256
256
256
256
256
248
256
444
700
700
256
256
256
256
256
248
692
692
444
256
256
240
256
256
752
700
700
240
256
248
256
256
444
198
256
248
240
256
256
256
248
700
444
216
256
256
256
256
852
240
252
256
256
248
240
776
444
444
256
256
256
256
240
444
444
256
256
240
256
752
256
256
144
444
692
444
780
444
444
256
248
248
256
256
256
768
700
760
256
256
444
700
792
588
420
700
256
248
248
248
254
256
256
700
768
248
852
788
692


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 891/891 [00:01<00:00, 775.47it/s]

256
256
248
256
256
256
256
218
256
240
780
700
444
756
852
256
256
252
256
256
716
444
444
444
256
716
692
256
256
256
208
256
256
256
768
444
692
444
700
444
444
692
439
692
436
256
248
256
256
248
752
752
256
768
248
256
256
444
700
700
700
444
700
784
256
248
248
756
720
256
256
256
700
152
256
444
700
256
756
256
236
252
248
256
248
248
256
256
256
256
421
700
588
256
700
700
256
246
256
256
248
256
256
240
256
852
780
206
852
256
780
700
444
256
256
256
248
256
240
218
256
240
444
444
428
256
444
588
752
256
242
232
256
776
256
256
752
444
444


In [160]:
ifc.write('../added.ifc') 

In [164]:
viewer = JupyterIFCRenderer(ifc, size=(400,300))
viewer

/home/haritha/miniconda3/envs/cdbb/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [ ]:
## meshlab downsampling -DOESNT WORK

In [125]:

print('input mesh has', mesh.vertex_number(), 'vertex and', mesh.face_number(), 'faces')


#Estimate number of faces to have 100+10000 vertex using Euler

#Simplify the mesh. Only first simplification will be agressive
#ms.apply_filter('simplification_quadric_edge_collapse_decimation', targetfacenum=numFaces, preservenormal=False)
ms.apply_filter('simplification_clustering_decimation', threshold=ml.Percentage(5))
print("Decimated to", numFaces, "faces mesh has", ms.current_mesh().vertex_number(), "vertex")


m = ms.current_mesh()
print('output mesh has', m.vertex_number(), 'vertex and', m.face_number(), 'faces')
ms.save_current_mesh('../output.ply')

input mesh has 768 vertex and 256 faces
Decimated to 30 faces mesh has 80 vertex
output mesh has 80 vertex and 152 faces


/tmp/ipykernel_2652/2845713468.py:9: UserWarning: Deprecated Filter Warning: You should use 'meshing_decimation_clustering' instead of 'simplification_clustering_decimation'. See https://pymeshlab.readthedocs.io/en/latest/index.html#filters-renaming
  ms.apply_filter('simplification_clustering_decimation', threshold=ml.Percentage(5))


In [131]:
print(m.face_matrix())

[[57 24 76]
 [24  4 76]
 [20  4 24]
 [20  6  4]
 [ 5  6 20]
 [ 5  2  6]
 [ 8  1  2]
 [ 8  7  1]
 [ 7  9  0]
 [74  0  9]
 [74 12  0]
 [10 30 12]
 [11 68 31]
 [71 50 40]
 [77 78 50]
 [78 58 50]
 [78 49 58]
 [78 72 49]
 [72 39 49]
 [72 70 39]
 [65 66 38]
 [57 76  3]
 [51  0 12]
 [51  2  1]
 [ 7  0  1]
 [51  6  2]
 [51  4  6]
 [51 76  4]
 [51  3 76]
 [67 31 30]
 [51 66  3]
 [51 38 66]
 [50 58 51]
 [50 51 40]
 [40 51 31]
 [74 10 12]
 [79 24 57]
 [79 20 24]
 [ 5  8  2]
 [18 53 37]
 [11 79 68]
 [45 23 35]
 [46 13 64]
 [36 75 34]
 [70 38 39]
 [39 51 49]
 [37 28 13]
 [41 46 64]
 [51  1  0]
 [18 37 13]
 [21 13 29]
 [46 56 13]
 [65 62 66]
 [79  5 20]
 [68 71 40]
 [36 17 13]
 [32 37 60]
 [36 13 75]
 [17 15 23]
 [43 44 55]
 [13 44 69]
 [62  3 66]
 [75 13 73]
 [68 40 31]
 [26 33 14]
 [26 61 63]
 [42 26 16]
 [32 60 26]
 [25 26 53]
 [36 34 15]
 [57  3 62]
 [79 77 71]
 [26 15 34]
 [73 16 34]
 [48 13 47]
 [25 22 26]
 [19 13 28]
 [22 59 26]
 [21 27 13]
 [48 18 13]
 [26 63 33]
 [54 61 26]
 [43 69 44]
 [26

In [134]:
shape.CoordIndex = m.face_matrix().tolist()
shape.Coordinates.CoordList = m.vertex_matrix().tolist()